<a href="https://colab.research.google.com/github/kolinz/notebook4AI/blob/main/Granite4_0_ja_dataset_FT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth your local device, follow [our guide](https://docs.unsloth.ai/get-started/install-and-update). This notebook is licensed [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


このファインチューニング用ノートブックのもとになったノートブックは「[Granite4.0_350M.ipynb](https://github.com/unslothai/notebooks/blob/main/nb/Granite4.0_350M.ipynb)」です。

### 日本語データセットのあるGoogle ドライブに接続。

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### UnslothとMambaのインストール

In [2]:
%%capture
import os, importlib.util
!pip install --upgrade -qqq uv
if importlib.util.find_spec("torch") is None or "COLAB_" in "".join(os.environ.keys()):
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    !uv pip install -qqq \
        "torch==2.7.1" "triton>=3.3.0" {get_numpy} {get_pil} torchvision bitsandbytes "transformers==4.56.2" \
        "unsloth-zoo==2025.12.7" \
        "unsloth==2025.12.9"
elif importlib.util.find_spec("unsloth") is None:
    !uv pip install -qqq unsloth
!uv pip install --upgrade --no-deps transformers==4.56.2 tokenizers trl==0.22.2 unsloth unsloth_zoo

# These are mamba kernels and we must have these for faster training
# Mamba kernels are for now supported only on torch==2.7.1. If you have newer torch versions, please wait 30 minutes for it to compile
!uv pip install --no-build-isolation mamba_ssm==2.2.5
!uv pip install --no-build-isolation causal_conv1d==1.5.2

### Unslothのモデルをダウンロード

In [3]:
from unsloth import FastLanguageModel
import torch
print("ファインチューニングのベースになるモデルの取得")

fourbit_models = [
    "unsloth/granite-4.0-micro",
    "unsloth/granite-4.0-h-micro",
    "unsloth/granite-4.0-h-tiny",
    "unsloth/granite-4.0-h-small",

    # Base pretrained Granite 4 models
    "unsloth/granite-4.0-micro-base",
    "unsloth/granite-4.0-h-micro-base",
    "unsloth/granite-4.0-h-tiny-base",
    "unsloth/granite-4.0-h-small-base",

    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/Phi-4",
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/orpheus-3b-0.1-ft-unsloth-bnb-4bit" # [NEW] We support TTS models!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/granite-4.0-350m-bnb-4bit",
    max_seq_length = 2048,   # Choose any for long context!
    load_in_4bit = False,    # 4 bit quantization to reduce memory
    load_in_8bit = False,    # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/usr/local/lib/python3.12/dist-packages/torchao/quantization/quant_api.py:2525: SyntaxWarning: invalid escape sequence '\.'
  * regex for parameter names, must start with `re:`, e.g. `re:language\.layers\..+\.q_proj.weight`.


🦥 Unsloth Zoo will now patch everything to make training faster!
ファインチューニングのベースになるモデルの取得
==((====))==  Unsloth 2025.12.9: Fast Granitemoehybrid patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


model.safetensors:   0%|          | 0.00/705M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/170 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

ファインチューニング用のパラメータ設定

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",
                      "shared_mlp.input_linear", "shared_mlp.output_linear"],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Making `model.base_model.model.model` require gradients


<a name="Data"></a>
### データ準備
#### 📄 Googleドライブ上のCSVファイルを読み込む

お客様の問題を積極的にサポートし、問題解決を行うカスタマーサポートのためのモデルを作ります。

サンプルは、以下の2つの列を持つCSVファイルを読み込み、ファインチューニングに用います。

- **input**: カスタマーサポートとの短いやり取り
- **recommendation**: 問題の解決方法に関する提案

---
<br>

granite-4 のチャット テンプレートは次のようになります。
```
<|start_of_role|>system<|end_of_role|>Knowledge Cutoff Date: April 2024.
Today's Date: June 24, 2025.
You are Granite, developed by IBM. You are a helpful AI assistant.<|end_of_text|>

<|start_of_role|>user<|end_of_role|>How do astronomers determine the original wavelength of light emitted by a celestial body at rest, which is necessary for measuring its speed using the Doppler effect?<|end_of_text|>

<|start_of_role|>assistant<|end_of_role|>Astronomers make use of the unique spectral fingerprints of elements found in stars...<|end_of_text|>
```

日本語データセットの読み込みを実施

In [5]:
from datasets import load_dataset

# 1. CSVの読み込み
sheet_path = '/content/drive/MyDrive/finetune_support_ja_100_rich.csv'
dataset = load_dataset(
    "csv",
    data_files={"train": sheet_path},
    column_names=["input", "recommendation"],
    skiprows=1
)["train"]

# 2. モデルが理解できる形式に変換する関数
def formatting_prompts_func(examples):
    inputs  = examples["input"]
    outputs = examples["recommendation"]
    texts = []
    for input_text, output_text in zip(inputs, outputs):
        # Granite Baseモデルが「対話」として認識できるタグを付けます
        # input自体に "User:" が含まれているので、それを活かしつつ構成します
        text = f"<|user|>\n{input_text}\n<|assistant|>\n{output_text}\n<|endoftext|>"
        texts.append(text)
    return { "text" : texts }

# データセットに適用
dataset = dataset.map(formatting_prompts_func, batched = True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/105 [00:00<?, ? examples/s]

csv 形式のデータセットとして読み込みましたが、以下のように会話形式にフォーマットし、チャット テンプレートを適用する必要があります。

```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

helper function `formatting_prompts_func` を使用します。

In [6]:
def formatting_prompts_func(examples):
    user_texts = examples['input']
    response_texts = examples['recommendation']
    messages = [
        [{"role": "user", "content": user_text},
        {"role": "assistant", "content": response_text}] for user_text, response_text in zip(user_texts, response_texts)
    ]
    texts = [tokenizer.apply_chat_template(message, tokenize = False, add_generation_prompt = False) for message in messages]

    return { "text" : texts, }

# Explicitly set the chat template for the tokenizer
DEFAULT_CHAT_TEMPLATE = """{% set system_message = 'You are a helpful assistant. Please ensure responses are professional, accurate, and safe.' %}
{% set loop_messages = messages %}

{% if messages %}
    {% if messages[0]['role'] == 'system' %}
        {% set loop_messages = messages[1:] %}
        {% set system_message = messages[0]['content'] %}
    {% endif %}
{% endif %}

{{ '<|start_of_role|>system<|end_of_role|>' + system_message + '<|end_of_text|>' }}
{% for message in loop_messages %}
    {{ '<|start_of_role|>' + message['role'] + '<|end_of_role|>' + message['content'] + '<|end_of_text|>' }}
{% endfor %}
{% if add_generation_prompt %}
    {{ '<|start_of_role|>assistant<|end_of_role|>' }}
{% endif %}"""
tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE

dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/105 [00:00<?, ? examples/s]

ここで、フォーマット前の生の入力データを確認します。

In [7]:
dataset[5]["input"]

'User: ログインしようとすると画面が真っ白になります。\nAgent: ご不便をおかけして申し訳ありません。発生するタイミングを教えてください。\nUser: IDとパスワードを入力してログインを押した直後です。\nAgent: ご利用のブラウザと端末を教えていただけますか？'

In [8]:
dataset[5]['recommendation']

'ログイン後に画面が表示されない問題が発生している。\nブラウザ依存やキャッシュ不整合の可能性があるため、利用環境を確認したうえで切り分けを行うことが重要である。'

チャット テンプレートがこれらの会話をどのように変えたかがわかります。

In [9]:
dataset[5]["text"]

'\n\n<|start_of_role|>system<|end_of_role|>You are a helpful assistant. Please ensure responses are professional, accurate, and safe.<|end_of_text|>\n    <|start_of_role|>user<|end_of_role|>User: ログインしようとすると画面が真っ白になります。\nAgent: ご不便をおかけして申し訳ありません。発生するタイミングを教えてください。\nUser: IDとパスワードを入力してログインを押した直後です。\nAgent: ご利用のブラウザと端末を教えていただけますか？<|end_of_text|>\n    <|start_of_role|>assistant<|end_of_role|>ログイン後に画面が表示されない問題が発生している。\nブラウザ依存やキャッシュ不整合の可能性があるため、利用環境を確認したうえで切り分けを行うことが重要である。<|end_of_text|>\n'

<a name="Train"></a>
### モデルの学習
モデルをトレーニングしてみましょう。高速化のため60ステップで実行しますが、フル実行する場合は「num_train_epochs=1」に設定し、「max_steps=None」をオフにすることもできます。

トレーニングロスが、nan になる場合は、こちらのパラメータと、ベースになるモデルのパラメータサイズ、学習用のデータの３つを見直す必要があります。

In [11]:
from trl import SFTTrainer, SFTConfig
from transformers import TrainingArguments
import torch # Make sure torch is imported to use torch.cuda.is_bf16_supported

# Removed 'from unsloth.util import is_bfloat16_supported' as it's deprecated/removed.
# Using torch.cuda.is_bf16_supported() directly.

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 60,
        learning_rate = 2e-4,
        max_grad_norm = 0.5,
        logging_steps = 1,
        optim = "adamw_torch",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(), # A100/30系なら絶対bf16が安定
    ),
)

Unsloth: We found double BOS tokens - we shall remove one automatically.


Unslothのtrain_on_completionsメソッドを使用して、アシスタントの出力のみで学習し、ユーザーの入力による損失を無視します。これにより、ファインチューンの精度が向上します。

In [12]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_of_role|>user|end_of_role|>",
    response_part = "<|start_of_role|>assistant<|end_of_role|>",
)

Map (num_proc=6):   0%|          | 0/105 [00:00<?, ? examples/s]

命令部分のマスクができたことを確認しましょう！もう一度5行目を出力してみましょう。

In [13]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'\n\n<|start_of_role|>system<|end_of_role|>You are a helpful assistant. Please ensure responses are professional, accurate, and safe.<|end_of_text|>\n    <|start_of_role|>user<|end_of_role|>User: ログインしようとすると画面が真っ白になります。\nAgent: ご不便をおかけして申し訳ありません。発生するタイミングを教えてください。\nUser: IDとパスワードを入力してログインを押した直後です。\nAgent: ご利用のブラウザと端末を教えていただけますか？<|end_of_text|>\n    <|start_of_role|>assistant<|end_of_role|>ログイン後に画面が表示されない問題が発生している。\nブラウザ依存やキャッシュ不整合の可能性があるため、利用環境を確認したうえで切り分けを行うことが重要である。<|end_of_text|>\n'

次に、マスクされた例を出力してみましょう。答えだけが表示されるはずです。

In [15]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[5]["labels"]]).replace(tokenizer.pad_token, " ")

'                                                                                                                                                 ログイン後に画面が表示されない問題が発生している。\nブラウザ依存やキャッシュ不整合の可能性があるため、利用環境を確認したうえで切り分けを行うことが重要である。<|end_of_text|>\n'

In [16]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
0.723 GB of memory reserved.


モデルをトレーニングしましょう！トレーニング実行を再開するには、trainer.train(resume_from_checkpoint = True) を設定します。

```
Mambaカーネルのコンパイルには約10分かかる場合がありますのでご注意ください。しばらくお待ちください。
```

In [17]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 105 | Num Epochs = 5 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 13,303,808 of 365,683,712 (3.64% trained)


Step,Training Loss
1,2.188500
2,2.070000
3,1.763500
4,1.964500
5,1.698900
6,1.530500
7,1.306400
8,1.538900
9,1.181600
10,1.170800


In [18]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

107.5626 seconds used for training.
1.79 minutes used for training.
Peak reserved memory = 1.316 GB.
Peak reserved memory for training = 0.593 GB.
Peak reserved memory % of max memory = 8.927 %.
Peak reserved memory for training % of max memory = 4.023 %.


<a name="推論"></a>
### 推論
Unslothネイティブ推論を使ってモデルを実行してみましょう！学習内容を把握するために、トレーニングデータには含まれていないサンプル入力をいくつか使用します。

In [19]:
# @title Test Scenarios
# --- Scenario 1: Video-Conferencing Screen-Share Bug (11 turns) ---
scenario_1 = """
User: 会議の参加者全員、私が画面共有をすると真っ暗な画面しか見えないようです。
Agent: ご不便をおかけして申し訳ありません。ウィンドウを共有していますか、それとも画面全体を共有していますか？
User: macOS Sonomaで画面全体を共有しています。
Agent: ありがとうございます。「設定」→「ビデオ」にある「ハードウェアアクセラレーションを有効にする」のスイッチはオンになっていますか？
User: はい、そのスイッチはオンになっています。
Agent: 一度それをオフに切り替えてから、クイックテスト共有を試していただけますか？
User: 試しましたが、参加者にはまだ真っ暗に見えるようです。
Agent: 承知いたしました。デスクトップアプリのv5.4.2をお使いですか、それともブラウザ版をご利用でしょうか？
User: デスクトップ版のv5.4.2です。ちょうど今朝アップデートしたばかりです。
"""

# --- Scenario 2: Smart-Lock Low-Battery Loop (9 turns) ---
scenario_2 = """
User: 電池を交換したのですが、ロックアプリの表示が5%のままで、オートロックが作動しません。
Agent: ファームウェアを確認しましょう。アプリの「設定」→「デバイス情報」に進み、どのバージョンが表示されているか教えていただけますか？
User: 3.18.0-alphaです。
Agent: 最新の安定版は3.17.5です。ベータプログラムに登録されていますか？
User: 数ヶ月前に登録したかもしれません。
Agent: ベータ版のビルドでは、稀にバッテリー残量が誤報されることがあります。電池を一度抜き、10秒待ってから再挿入して、LEDの点滅パターンを確認してください。
User: LEDが青に2回点滅した後、赤に1回点滅しました。
Agent: その点滅コードは「設定の不一致（config mismatch）」を意味します。お手元に古い方の電池はまだ残っていますか？
User: もう捨ててしまいました。
"""

# --- Scenario 3: Accounting SaaS — Corrupted Invoice Export (10 turns) ---
scenario_3 = """
User: 今日ダウンロードする請求書が、どれも白紙のPDFとして開いてしまいます。
Agent: それは過去の請求書、新しいもの、あるいは両方で発生していますか？
User: 両方です。書き出したものはすべて0バイトになります。
Agent: 「一括操作」から書き出していますか、それとも個別の請求書ページからですか？
User: 個別のページからです。
Agent: ブラウザとOSの組み合わせは何でしょうか？
User: Windows 11上のChromeです。最新の状態にアップデートしています。
Agent: UTCの午前10時に新しいPDFレンダラーをリリースしました。キャッシュの不具合を除外するために、一度Edgeで試していただけますか？
User: Edgeで試しましたが、やはり0バイトのファイルになります。
"""

In [20]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": scenario_1},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    padding = True,
    return_tensors = "pt",
    return_dict=True,
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = False)

_ = model.generate(**inputs,
                   streamer = text_streamer,
                   max_new_tokens = 512, # Increase if tokens are getting cut off
                   use_cache = True,
                   # Adjust the sampling params to your preference
                   do_sample=True,
                   temperature = 0.7, top_p = 0.8, top_k = 20,
)



<|start_of_role|>system<|end_of_role|>You are a helpful assistant. Please ensure responses are professional, accurate, and safe.<|end_of_text|>
    <|start_of_role|>user<|end_of_role|>
User: 会議の参加者全員、私が画面共有をすると真っ暗な画面しか見えないようです。
Agent: ご不便をおかけして申し訳ありません。ウィンドウを共有していますか、それとも画面全体を共有していますか？
User: macOS Sonomaで画面全体を共有しています。
Agent: ありがとうございます。「設定」→「ビデオ」にある「ハードウェアアクセラレーションを有効にする」のスイッチはオンになっていますか？
User: はい、そのスイッチはオンになっています。
Agent: 一度それをオフに切り替えてから、クイックテスト共有を試していただけますか？
User: 試しましたが、参加者にはまだ真っ暗に見えるようです。
Agent: 承知いたしました。デスクトップアプリのv5.4.2をお使いですか、それともブラウザ版をご利用でしょうか？
User: デスクトップ版のv5.4.2です。ちょうど今朝アップデートしたばかりです。
<|end_of_text|>
    <|start_of_role|>assistant<|end_of_role|>
画面共有機能の設定に関する問い合わせ。画面全体のみを共有するか、ユーザー側の設定（アプリ設定）を確認することが重要である。<|end_of_text|>


In [21]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": scenario_2},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    padding = True,
    return_tensors = "pt",
    return_dict=True,
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = False)

_ = model.generate(**inputs,
                   streamer = text_streamer,
                   max_new_tokens = 512, # Increase if tokens are getting cut off
                   use_cache = True,
                   # Adjust the sampling params to your preference
                   do_sample=False,
                   temperature = 0.7, top_p = 0.8, top_k = 20,
)



<|start_of_role|>system<|end_of_role|>You are a helpful assistant. Please ensure responses are professional, accurate, and safe.<|end_of_text|>
    <|start_of_role|>user<|end_of_role|>
User: 電池を交換したのですが、ロックアプリの表示が5%のままで、オートロックが作動しません。
Agent: ファームウェアを確認しましょう。アプリの「設定」→「デバイス情報」に進み、どのバージョンが表示されているか教えていただけますか？
User: 3.18.0-alphaです。
Agent: 最新の安定版は3.17.5です。ベータプログラムに登録されていますか？
User: 数ヶ月前に登録したかもしれません。
Agent: ベータ版のビルドでは、稀にバッテリー残量が誤報されることがあります。電池を一度抜き、10秒待ってから再挿入して、LEDの点滅パターンを確認してください。
User: LEDが青に2回点滅した後、赤に1回点滅しました。
Agent: その点滅コードは「設定の不一致（config mismatch）」を意味します。お手元に古い方の電池はまだ残っていますか？
User: もう捨ててしまいました。
<|end_of_text|>
    <|start_of_role|>assistant<|end_of_role|>
電池交換後のロックアプリ表示まで改善するための具体的な設定手順を説明し、ユーザーの不安を軽減する対応が求められる。<|end_of_text|>


<a name="Save"></a>
### モデルの保存と読み込み
モデルをLoRAアダプターとして保存するには、オンライン保存の場合はHuggingfaceの`push_to_hub`、ローカル保存の場合は`save_pretrained`を使用してください。

**[注意]** これはLoRAアダプターのみを保存し、完全なモデルは保存しません。16ビットまたはGGUF形式で保存するには、下にスクロールしてください。

In [24]:
model.save_pretrained("granite4_ft_model")  # Local saving , デフォルトモデル名 lora_model
tokenizer.save_pretrained("granite4_ft_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('granite4_ft_model/tokenizer_config.json',
 'granite4_ft_model/special_tokens_map.json',
 'granite4_ft_model/chat_template.jinja',
 'granite4_ft_model/vocab.json',
 'granite4_ft_model/merges.txt',
 'granite4_ft_model/added_tokens.json',
 'granite4_ft_model/tokenizer.json')

推論用に保存した LoRA アダプターをロードする場合は、False を True に設定します。

In [25]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "granite4_ft_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
    )

messages = [
    {"role": "user", "content": scenario_3},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    padding = True,
    return_tensors = "pt",
    return_dict=True,
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = False)

_ = model.generate(**inputs,
                   streamer = text_streamer,
                   max_new_tokens = 512, # Increase if tokens are getting cut off
                   use_cache = True,
                   # Adjust the sampling params to your preference
                   do_sample=False,
                   temperature = 0.7, top_p = 0.8, top_k = 20,
)



<|start_of_role|>system<|end_of_role|>You are a helpful assistant. Please ensure responses are professional, accurate, and safe.<|end_of_text|>
    <|start_of_role|>user<|end_of_role|>
User: 今日ダウンロードする請求書が、どれも白紙のPDFとして開いてしまいます。
Agent: それは過去の請求書、新しいもの、あるいは両方で発生していますか？
User: 両方です。書き出したものはすべて0バイトになります。
Agent: 「一括操作」から書き出していますか、それとも個別の請求書ページからですか？
User: 個別のページからです。
Agent: ブラウザとOSの組み合わせは何でしょうか？
User: Windows 11上のChromeです。最新の状態にアップデートしています。
Agent: UTCの午前10時に新しいPDFレンダラーをリリースしました。キャッシュの不具合を除外するために、一度Edgeで試していただけますか？
User: Edgeで試しましたが、やはり0バイトのファイルになります。
<|end_of_text|>
    <|start_of_role|>assistant<|end_of_role|>
複数の請求書がブラウザやOSの不具合を招いて0バイトのPDFになっていることが発見されている。
対応ブラウザ（例：Chrome）やOS（例：Windows 11）の具体的不具合を除去するための最新のPDFレンダラーのリリースを案内する。<|end_of_text|>


### モデルのエクスポート

In [26]:
# 8ビット量子化を指定
if True:
    model.save_pretrained_gguf("export_model", tokenizer, quantization_method = "q8_0")

Unsloth: Merging model weights to 16-bit format...
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...


Unsloth: Copying 1 files from cache to `export_model`: 100%|██████████| 1/1 [00:08<00:00,  8.88s/it]


Successfully copied all 1 files from cache to `export_model`
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [00:04<00:00,  4.80s/it]


Unsloth: Merge process complete. Saved to `/content/export_model`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Cloning llama.cpp repository
Unsloth: Install GGUF and other packages
Unsloth: Successfully installed llama.cpp!
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into f16 GGUF format.
This might take 3 minutes...
Unsloth: Initial conversion completed! Files: ['granite-4.0-350m.F16.gguf']
Unsloth: [2]

### モデル名の変更とダウンロード

「granite-4.0-350m.Q8_0.gguf」としてで保存されているので、ファインチューニングしたことがわかりやすいように、「granite-4.0-350m-FT-ja.Q8_0.gguf」にモデル名を変更し、Google Colabからダウンロードできるようにする。

In [28]:
import os

src = "/content/granite-4.0-350m.Q8_0.gguf"
dst = "/content/granite-4.0-350m-FT-ja.Q8_0.gguf"

os.rename(src, dst)

In [29]:
from google.colab import files
files.download('/content/granite-4.0-350m-FT-ja.Q8_0.gguf')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>